In [1]:
import json
import math

In [59]:
with open('caption/entity_dataset_train_idf.json', 'r') as f:
    data = json.load(f)

In [20]:
def compute_idf(data):
    des_count = 0
    idx_count = {}
    # Count, des_count is N, idx_count is df
    for d in data:
        entities = d['entities']
        for entity in entities:
            for des in entity['entity_descs']:
                des_count += 1
                for idx in des['indexed']:
                    idx_count[idx] = idx_count.get(idx, 0) + 1
    # compute idf
    for idx in idx_count:
        idx_count[idx] = math.log(des_count / (idx_count[idx] + 1))
    return des_count, idx_count

In [21]:
des_count, idx_count = compute_idf(data)

In [22]:
idx_count

{0: 0.5561362239235808,
 1: 2.437461957006778,
 2: 3.4969170307415327,
 3: 3.30826552680355,
 10: 6.468361760407515,
 4: 7.392218732664986,
 5: 9.682225043452172,
 6: 9.41029132796853,
 13: 4.436188300646204,
 14: 10.152228672697907,
 7: 6.1074245560359435,
 8: 3.2023722176971354,
 9: 6.774641156674886,
 11: 6.757720279186549,
 12: 6.805839527530748,
 23: 4.443127036564107,
 24: 3.6476411427752518,
 15: 4.209954369780697,
 16: 4.913130665809843,
 17: 5.137601357332114,
 25: 2.2428126311241114,
 19: 6.622931288408437,
 18: 7.256316734426128,
 20: 3.284982937053981,
 21: 10.257589188355734,
 22: 8.604666163981895,
 26: 4.295153028628578,
 27: 9.746763564589743,
 28: 5.824130380049581,
 29: 4.455806552447999,
 32: 7.239878008082968,
 30: 9.12260925551675,
 31: 5.916673941924284,
 34: 8.035973157895356,
 33: 4.8132493244309815,
 35: 4.067729259622737,
 36: 5.008228666534229,
 37: 3.439272472020838,
 38: 8.206318523642594,
 39: 5.719033751449627,
 40: 7.5060538753137855,
 41: 5.277031349496

In [24]:
with open('caption/entity_train_idf.json', 'w') as f:
    f.write(json.dumps(idx_count))

In [3]:
with open('caption/entity_train_idf.json') as f:
    idf = json.load(f)

In [9]:
data[1]['entities'][0]['entity_descs'][0]

{'entity_desc_id': 2,
 'entity_desc': 'a machine',
 'token': ['a', 'machine'],
 'indexed': [0, 10]}

In [22]:
import numpy as np
new_data = []
for d in data:
    entities = d['entities']
    for entity in entities:
        tf_idf_scores = []
        for des in entity['entity_descs']:
            score = 0
            for idx in des['indexed']:
                score += idf.get(str(idx), 0)
            tf_idf_scores.append(score)
        arg = np.argsort(tf_idf_scores)
        for idx, des in enumerate(entity['entity_descs']):
            des['tf_idf_level'] = int(arg[idx])
    new_data.append(d)

In [62]:
new_data[1]['entities'][0]['entity_descs'][0]['tf_idf_level']==0

True

In [25]:
with open('caption/entity_dataset_train_idf.json', 'w') as f:
    f.write(json.dumps(new_data))

In [36]:
def fetch_easy_and_hard(data):
    easy = []
    hard = []
    for d in data:
        entities = d['entities']
        for entity in entities:
            descs = entity['entity_descs']
            if len(descs) <= 1:
                continue
            for des in descs:
                if des['tf_idf_level'] == 0:
                    easy.extend(des['token'])
                if des['tf_idf_level'] == len(descs) - 1:
                    hard.extend(des['token'])
                    
    import collections
    easy_dict = collections.Counter(easy)
    hard_dict = collections.Counter(hard)
    return easy_dict, hard_dict

In [52]:
easy, hard = fetch_easy_and_hard(data)

In [51]:
# easy = sorted(easy.items(), key=lambda x: x[1], reverse=True)
# hard = sorted(hard.items(), key=lambda x: x[1], reverse=True)
# diff = set([item[0] for item in hard]) - set([item[0] for item in easy])

AttributeError: 'list' object has no attribute 'items'

In [58]:
sorted(diff, key=lambda x: hard[x], reverse=True)

['hispanic',
 'excited',
 'inside',
 'facial',
 'apparatus',
 'own',
 'scientist',
 'medium',
 'both',
 'beater',
 'to',
 'mule',
 'with',
 'is',
 'mustached',
 'sleeves',
 'deserted',
 'rolled',
 'shady',
 'maker',
 'move',
 'lush',
 'vast',
 'hanging',
 'mature',
 'muslim',
 'asphalt',
 'opened',
 'rocking',
 'themed',
 'approaching',
 'metallic',
 'merchant',
 'scruffy',
 'masked',
 'rectangular',
 'carved',
 'hardwood',
 'trick',
 'goofy',
 'volkswagen',
 'index',
 'as',
 'smock',
 'workman',
 'mats',
 'wears',
 'lays',
 'wrestlers',
 'dilapidated',
 'late',
 'folded',
 'quarter',
 'plow',
 'contestant',
 'passerby',
 'enthusiastic',
 'oar',
 'shotgun',
 'straight',
 'size',
 'cutoff',
 'blackboard',
 'unpaved',
 'messenger',
 'horseback',
 'handkerchief',
 'blank',
 'rest',
 'fleece',
 'obese',
 'smoking',
 'vivid',
 'irish',
 'ten',
 'splits',
 'overturned',
 'marked',
 'fourth',
 'pasture',
 'shoeshine',
 'rapid',
 'darker',
 'grilled',
 'advertising',
 'indoors',
 'passes',
 'g

In [56]:
diff